In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from flaml.autogen.agentchat2.llm_chat import (
    LLMChatMessage,
    LLMChatAgent,
)
from flaml.autogen.agentchat2.stream import ListMessageStream

In [11]:
stream = ListMessageStream()

In [12]:
debbie = LLMChatAgent(
    name="Debbie",
    address="debbie",
    message_stream=stream,
    system_message={
        "role": "system",
        "content": "You are debating with Jake whether it is better to travel to Yellow Stone from Seattle by car or by flight. Your position is that car is better.",
    },
    llm_config={"temperature": 1.0, "model": "gpt-3.5-turbo"},
)
jake = LLMChatAgent(
    name="Jake",
    address="jake",
    message_stream=stream,
    system_message={
        "role": "system",
        "content": "You are debating with user whether it is better to travel to Yellow Stone from Seattle by car or by flight. Your position is that flight is better.",
    },
    llm_config={"temperature": 1.0, "model": "gpt-3.5-turbo"},
    trigger=lambda message, context: len(context.chat_history) < 5,
)

2023-08-05 03:02:03,175 DEBUG Add subscriber <flaml.autogen.agentchat2.llm_chat.LLMChatAgent object at 0x7fcc05bb36d0> to address debbie
2023-08-05 03:02:03,178 DEBUG Add subscriber <flaml.autogen.agentchat2.llm_chat.LLMChatAgent object at 0x7fcc40788590> to address jake


In [13]:
stream.send("debbie", LLMChatMessage("Jake", "jake", "Debbie", "debbie", {"role": "user", "content": "Hello", "name": "Jake"}), lambda address, agent: address != "jake")

2023-08-05 03:02:03,207 DEBUG Send
{'message': {'content': 'Hello', 'name': 'Jake', 'role': 'user'},
 'receiver_address': 'debbie',
 'receiver_name': 'Debbie',
 'sender_address': 'jake',
 'sender_name': 'Jake'}
to address debbie


In [14]:
stream.run()

2023-08-05 03:02:03,247 DEBUG Send
{'message': {'content': "Hey Jake! How are you doing? I heard you're planning "
                        'a trip to Yellowstone. That sounds exciting! Have you '
                        'decided on the mode of transportation yet?',
             'name': 'Debbie',
             'role': 'user'},
 'receiver_address': 'jake',
 'receiver_name': 'Jake',
 'sender_address': 'debbie',
 'sender_name': 'Debbie'}
to address jake
2023-08-05 03:02:03,261 DEBUG Send
{'message': {'content': "Jake:I'm doing great, Debbie! Yes, I'm super excited "
                        'about the trip to Yellowstone. Thanks for asking. '
                        "I've been discussing it with a few friends, and we're "
                        'still debating whether to go by car or by flight. '
                        'What are your thoughts? Do you have any '
                        'recommendations?',
             'name': 'Jake',
             'role': 'user'},
 'receiver_address': 'debbi

In [16]:
jake.get_contexts()[0].chat_history

[{'role': 'user',
  'content': "Hey Jake! How are you doing? I heard you're planning a trip to Yellowstone. That sounds exciting! Have you decided on the mode of transportation yet?",
  'name': 'Debbie'},
 {'role': 'assistant',
  'content': "Jake:I'm doing great, Debbie! Yes, I'm super excited about the trip to Yellowstone. Thanks for asking. I've been discussing it with a few friends, and we're still debating whether to go by car or by flight. What are your thoughts? Do you have any recommendations?",
  'name': 'Jake'},
 {'role': 'user',
  'content': "Debbie: That's great, Jake! Yellowstone is a stunning destination. Personally, I would recommend traveling by car for a few reasons. First and foremost, traveling by car allows you to have more flexibility and control over your itinerary. You can take detours, explore scenic routes, and even make unplanned stops along the way. This gives you the opportunity to discover hidden gems and enjoy the journey as much as the destination.\n\nAddi

In [17]:
debbie.get_contexts()[0].chat_history

[{'role': 'user', 'content': 'Hello', 'name': 'Jake'},
 {'role': 'assistant',
  'content': "Hey Jake! How are you doing? I heard you're planning a trip to Yellowstone. That sounds exciting! Have you decided on the mode of transportation yet?",
  'name': 'Debbie'},
 {'role': 'user',
  'content': "Jake:I'm doing great, Debbie! Yes, I'm super excited about the trip to Yellowstone. Thanks for asking. I've been discussing it with a few friends, and we're still debating whether to go by car or by flight. What are your thoughts? Do you have any recommendations?",
  'name': 'Jake'},
 {'role': 'assistant',
  'content': "Debbie: That's great, Jake! Yellowstone is a stunning destination. Personally, I would recommend traveling by car for a few reasons. First and foremost, traveling by car allows you to have more flexibility and control over your itinerary. You can take detours, explore scenic routes, and even make unplanned stops along the way. This gives you the opportunity to discover hidden ge